In [1]:
%time
%load_ext autotime
%load_ext autoreload
%autoreload 2

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.5 µs


In [2]:
# typically takes 30 secs
from nlp_metrics import *

time: 20.8 s


In [3]:
database = load_pickle('../big_data/database2.pickle')
blocklist= ['CANNOT_DETECT','-PRON-','boil','sheet']
database = list(set([word for word in database if word not in blocklist]))
print(len(database))

2002
time: 26.7 ms


In [4]:
for line in database:
    print(line)

kopi
jahe
pinolillo
throttle
suneli
tsai
passoã
chūkadon
appenzeller
afternoon
moringa
candlenut
ratafia
tomatillo
cashew
jatobá
shell
aioli
classic
mastica
slivovitz
sprite
yuanyang
colada
kiwano
hazelnut
ugu
orangelo
dimmi
lan
qi
pulasan
vinaigrette
ficelle
bourbon
marnier
quince
cannelloni
spice
zucchini
khubz
nihamanchi
pashofa
chaya
wakame
kurrat
blackwell
mombin
soppressata
bammy
matzo
cracker
nesquik
vesper
dabu
pucko
demänovka
trout
condiment
liqueur
obstwasser
orzo
cabidela
tempeh
breath
tequila
cassia
mahi
taglierini
horilka
pilaf
jocote
bolani
moco
cantelope
fillet
amazake
jiuqu
chokecherry
falukorv
beech
liquor
kerabu
choy
farfalle
disaronno
nopal
trenette
ginkgo
fusi
jellyfish
d'ulivo
fennel
anelli
fogg
skyr
da
singani
tzatziki
guar
grapèro
arrival
awamori
tauco
chass
cosmopolitan
lettuce
pap
granadilla
lupin
caltrop
sweetsop
plum
boysenberry
phoron
fry
tinda
tomato
powder
blazer
phitti
blenheim
rabo
kukk
armagnac
calamansi
almondmilk
creamer
sorel
mokatika
fufu
mutton
sho

In [5]:
class ev(evaluation):
    def __init__(self, filename, tag):
        self.dic = self.load_dic({}, filename, tag)
        self.ori = tag
        self.gens = []

    def good(self):
        indice = []
        for i, v in tqdm.tqdm(self.dic.items()):
            true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
            true, pred = self.ingr(true), self.ingr(pred)
            scores = metrics(true, pred)
            scores.f1()
            
            ori_instr, gen_instr = v['%s_instr'%(self.ori)], v['%s_instr'%(self.gen)]
            score = 1-self.norm_dist(ori_instr, gen_instr, stem_only = False)
            
            if scores.f1() * score > 0.6:
                indice.append(i)
                print(v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)])
                print(v['%s_instr'%(self.ori)], v['%s_instr'%(self.gen)])
        return indice 
    
    def ingr_precision_recall(self, generate = 'ingr'):
        assert generate in ['ingr','instr','human']
        precision, recall = [], []
        for i, v in tqdm.tqdm(self.dic.items()):
            if generate == 'ingr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_ingr'%(self.gen)]
                true, pred = self.ingr(true), self.ingr(pred)
            elif generate == 'instr':
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.gen)]
                true, pred = self.ingr(true), self.instr(pred)
            elif generate == 'human':   
                true, pred = v['%s_ingr'%(self.ori)], v['%s_instr'%(self.ori)]
                true, pred = self.ingr(true), self.instr(pred)
            scores = metrics(true, pred)
            precision.append(scores.precision())
            recall.append(scores.recall())
    
        print('precision', sum(precision)/len(precision))
        print('recall', sum(recall)/len(recall))
    
    def instr(self, directions):
        instr = sp.spacy(directions)
        root_instr = []
        for chunk in instr.noun_chunks:
            idx_rootnoun = chunk.end - 1
            str_rootnoun = instr[idx_rootnoun].lemma_
            if str_rootnoun in database:
                root_instr.append(str_rootnoun)
        return root_instr
    
    def ingr(self, lst):
        '''
        Args: lst: a list of ingredient names
        used when len(lst) must equal to root_match
        '''
        hl = [[{'text':x, 'highlight': None} for x in i.split(' ')] for i in lst]
        root_match = []
        for i, ingr in enumerate(lst):
            if ' ' not in ingr:
                hl[i][0]['highlight'] = 'wrong'
                doc = sp.spacy(ingr)
                root_match.append(doc[0].lemma_)
            else:
                phrase = 'Mix the %s and water.'%ingr
                doc = sp.spacy(phrase)
                
                last_chunk = None
                for chunk in doc.noun_chunks:
                    if chunk.text != 'water':
                        last_chunk = chunk
                if not last_chunk:
                    root_match.append('CANNOT_DETECT')
                else:
                    found = False
                    for j, word in enumerate(hl[i]):
                        if doc[last_chunk.end - 1].text in word['text']:
                            hl[i][j]['highlight'] = 'wrong' 
                            root_match.append(doc[last_chunk.end - 1].lemma_)
                            found = True
                            break
                    if not found:
                        root_match.append('CANNOT_DETECT')
                        
        assert len(root_match) == len(lst)
        return root_match

    '''
    exporting data
    '''
    def to_bleu(self):
        to_write = {'%s_i'%(self.ori):'',
                    '%s_i'%(self.gen):'',
                    '%s_d'%(self.ori):'',
                    '%s_d'%(self.gen):''}
        
        for i, v in self.dic.items():
            to_write['%s_i'%(self.ori)] += self.add_space(' $ '.join(v['%s_ingr'%(self.ori)]))+ ' $ \n'
            to_write['%s_i'%(self.gen)] += self.add_space(' $ '.join(v['%s_ingr'%(self.gen)])) + ' $ \n'
            
            to_write['%s_d'%(self.ori)] += self.add_space(v['%s_instr'%(self.ori)])+ '\n'
            to_write['%s_d'%(self.gen)] += self.add_space(v['%s_instr'%(self.gen)])+ '\n'
        
        for k, v in to_write.items():
            save('../../to_gpt2/generation_%s.txt'%(k), v ,overwrite = True)
        !eval {"perl multi-bleu.perl ../../to_gpt2/generation_%s_i.txt < ../../to_gpt2/generation_%s_i.txt" %(self.ori, self.gen)}
        !eval {"perl multi-bleu.perl ../../to_gpt2/generation_%s_d.txt < ../../to_gpt2/generation_%s_d.txt" %(self.ori, self.gen)}
    
        !eval {"rouge -f ../../to_gpt2/generation_%s_i.txt ../../to_gpt2/generation_%s_i.txt --avg"%(self.ori, self.gen)}
        !eval {"rouge -f ../../to_gpt2/generation_%s_d.txt ../../to_gpt2/generation_%s_d.txt --avg"%(self.ori, self.gen)}
        
        print()
    '''
    instruction evaluation
    '''
    def instr_tree(self, stem_only = False):
        value = []
        for i, v in tqdm.tqdm(self.dic.items()):
            ori_instr, gen_instr = v['%s_instr'%(self.ori)], v['%s_instr'%(self.gen)]
            score = self.norm_dist(ori_instr, gen_instr, stem_only = stem_only)
            value.append(score)
        avg = sum(value)/len(value)
        print(avg)
        return avg

    def state_f1_freq(self):
        value = []
        for i, v in tqdm.tqdm(self.dic.items()):
            true, pred = v['%s_instr'%(self.ori)], v['%s_instr'%(self.gen)]
            true, pred = sp.match_state(true), sp.match_state(pred)
            scores = metrics(true, pred)
            value.append(scores.f1_freq())
        avg = sum(value)/len(value)
        print(avg)
        return avg
    
    def verb_f1_freq(self):
        value = []
        for i, v in tqdm.tqdm(self.dic.items()):
            true, pred = v['%s_instr'%(self.ori)], v['%s_instr'%(self.gen)]
            true, pred = sp.instructions(true)[1], sp.instructions(pred)[1]
            scores = metrics(true, pred)
            value.append(scores.f1_freq())
        avg = sum(value)/len(value)
        print(avg)
        return avg
    
    '''
    cleaning data
    '''
    def remove_end(self, text):
        return text.replace('\n','').split('<')[0]
    
    def reverse(self, text):
        '''
        Important data cleaning before NY times parser
        '''
        # replace things in brace
        text = re.sub(r'\([^)]*\)', '', text)

        # remove space before punct
        text = re.sub(r'\s([?.!,"](?:\s|$))', r'\1', text)

        # remove consecutive spaces
        text = re.sub(' +',' ',text).strip()
        return text
    
    def reverse_list(self, listoftext):
        output = []
        for text in listoftext:
            rev = self.reverse(text)
            if rev:
                output.append(rev)
        return output
    
    def add_space(self, line):
        # add space before punct
        line = re.sub('([.,!?()])', r' \1 ', line)
        line = re.sub('\s{2,}', ' ', line)
        return line
    
    '''
    tree edit distance
    '''

    def str2tree(self, instr, stem_only):
        instr = [x for x in instr.split('. ') if x]
        instr = treemaker.sents2tree(instr)
        if stem_only:
            instr = stem(instr)
        n_nodes = sum([len(line['ingredient']) +1 for line in instr])
        return build_tree(instr), n_nodes

    def norm_dist(self, ori_instr, gen_instr, stem_only):
        '''
        Args: ori_instr: str
        Args: gen_instr: str
        '''
        ori_tree, ori_nodes = self.str2tree(ori_instr, stem_only = stem_only)
        gen_tree, gen_nodes = self.str2tree(gen_instr, stem_only = stem_only)
        tree_dist = tree_distance(ori_tree, gen_tree)
        normed = tree_dist/(ori_nodes+gen_nodes)
        return normed

time: 126 ms


In [6]:
data = ev('../../to_gpt2/recipe1M_1218/test/y/', 'ori')
data.append_dic('../../to_gpt2/generation_1220_k3_test/', '117M')
indices = data.good()

load ../../to_gpt2/recipe1M_1218/test/y/
load ../../to_gpt2/generation_1220_k3_test/


  1%|▏         | 50/4000 [00:28<42:04,  1.56it/s]

['cilantro', 'pepper', 'garlic', 'olive oil', 'tomatoes', 'feta cheese with dried basil and tomato', 'pasta', 'cheese'] ['garlic', 'cheese', 'cilantro', 'olive oil', 'salt and pepper', 'pasta', 'cherry tomatoes']
bring a large pot of lightly salted water to a boil. cook pasta in boiling water until done drain. return pasta to the pot. mix in olive oil, garlic, tomatoes, feta, and parmesan cheese. stir in cilantro and season with salt and pepper. serve warm. enjoy!  bring a large pot of lightly salted water to a boil. add pasta and cook for 8 to 10 minutes or until al dente drain. in a large skillet, heat oil over medium-high heat. saute garlic and tomatoes for 5 minutes. add pasta and toss with tomato mixture, feta cheese, cilantro, and pepper. cook for 5 minutes or until heated through. serve with parmesan cheese. 


  1%|▏         | 55/4000 [00:32<44:00,  1.49it/s]

['butter', 'gala apples', 'eggs', 'cranberries', 'cinnamon', 'yellow cake mix', 'philadelphia cream cheese', 'sugar'] ['cinnamon', 'eggs', 'philadelphia cream cheese', 'sugar', 'butter', 'smith apple', 'yellow cake mix']
heat oven to 350 degrees f. line 13x9-inch pan with foil, with ends of foil extending over sides. microwave butter in large microwaveable bowl on high 1 to 1-1/2 min. or until melted. add dry cake mix, 1 egg and cinnamon beat with mixer until blended. press 2/3 of the dough onto bottom of prepared pan. beat cream cheese and sugar with mixer until blended. add remaining egg mix well. spread over crust top with fruit. pinch small pieces of remaining dough between your fingers press lightly into cream cheese layer. bake 45 to 50 min. or until center is almost set. cool completely. refrigerate 1 hour. use foil handles to lift dessert from pan before cutting into bars.  heat oven to 350f. line 13x9-inch pan with foil, with ends of foil extending over sides. spray with cooki

...

['butter', 'white sugar', 'sweet rice flour', 'vanilla extract', 'eggs', 'milk', 'baking soda'] ['eggs', 'baking soda', 'white rice flour', 'milk', 'coconut extract', 'vanilla extract', 'white sugar', 'butter']
preheat oven to 350 degrees f (175 degrees c. spray a 9x13-inch glass baking dish with cooking spray. sift together the rice flour, sugar, and baking soda set aside. beat eggs with evaporated milk, coconut milk, and vanilla extract. add flour and butter beat until well combined. pour into prepared pan and bake in preheated oven for 30 minutes. allow to cool completely before cutting into small (1 1/2 inch squares.  preheat oven to 325 degrees f (165 degrees c. grease a 9x5 inch loaf pan. in a large bowl, mix together rice flour, baking soda, sugar, and butter until the mixture is crumbly. mix in eggs one at a time, beating well after each addition. stir in vanilla. pour into prepared pan. bake for 1 hour, until golden brown and a knife inserted into the center comes out clean. 


100%|█████████▉| 3983/4000 [34:42<00:10,  1.55it/s]

['zucchini', 'vanilla', 'semi sweet chocolate chips', 'eggs', 'virgin coconut oil', 'baking powder', 'cinnamon', 'unsweetened applesauce', 'whole wheat pastry flour', 'cocoa powder', 'salt', 'sugar', 'baking soda'] ['sugar', 'cocoa', 'baking soda', 'baking powder', 'eggs', 'zucchini', 'cinnamon', 'vanilla', 'unsweetened applesauce', 'salt', 'semi sweet chocolate chips', 'flour', 'vegetable oil']
preheat the oven to 350f. line muffin pans with liners (or grease well, set aside. in large bowl combine flour, cocoa, baking powder, baking soda, cinnamon and salt. in medium bowl whisk together sugar, eggs, applesauce, oil, and vanilla. stir in zucchini. add wet ingredients to dry ingredients. stir just until moistened. fold in chocolate chips. distribute batter evenly. bake 20-22 minutes or until a toothpick entered in center of muffins comes clean. cool 5 min on a cooling rack. yum!  preheat oven 350 fahrenheit. in a large bowl, combine the dry ingredients (flour through salt. in another bo

100%|██████████| 4000/4000 [34:50<00:00,  2.26it/s]

time: 34min 52s


In [7]:
len(indices)

67

time: 35.2 ms


In [9]:
data_1218 = load_pickle('../big_data/data_1218.pickle')

time: 7.84 s


In [12]:
import json

time: 46.1 ms


In [13]:
autofill = {i: v for i, v in enumerate(data_1218) if i in indices}

with open('../share_shuke/autofill_1218.json', 'w') as f:
    json.dump(autofill, f)

time: 898 ms
